# Synoptic types in Python: an analyis of circulation regimes over the New Zealand region

A definitive introduction to the concept of synoptic types (also *weather regimes*, *circulation regimes*) is beyond the scope of this session, again a good reference 
is [Michelangeli, P.-A., Vautard, R., & Legras, B. (1995). Weather Regimes: Recurrence and Quasi Stationarity. Journal of Atmospheric Sciences, 52, 1237–1256.](http://journals.ametsoc.org/doi/abs/10.1175/1520-0469%281995%29052%3C1237%3AWRRAQS%3E2.0.CO%3B2)

In short: 

+ there is day to day *variability* in the patterns of atmospheric circulation at all scales from regional to global   
        
        
+ this variability is NOT random 

        
+ at the regional scale particularly, some configuration are preferred --> "regimes"  

        
+ in the non-linear dynamics jargon, these preferred, recurrent configurations of atmospheric circulation are called "attractors basins"  


An illustration of the concept of *attractor basins* with the [Lorenz's system](https://en.wikipedia.org/wiki/Lorenz_system)
                                                               
Lorenz first discovered chaos by accident while developing a simple mathematical model of atmospheric convection, using three ordinary differential equations:

$$
\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}
$$

He found that nearly indistinguishable initial conditions could produce completely divergent outcomes, rendering weather prediction impossible beyond a time horizon of about a fortnight.

Now known as the **Lorenz System**, this model demonstrates chaos at certain parameter values, below is an animation of its solutions

This animation has been taken from the implementation of the Lorenz's system in Python, by Geoff Boeing, available at [https://github.com/gboeing/lorenz-system/blob/master/lorenz-system-attractor-animate.ipynb](https://github.com/gboeing/lorenz-system/blob/master/lorenz-system-attractor-animate.ipynb)

![](./images/animated-lorenz-attractor.gif)

In this case, there is clearly 2 **preferred regions** of the phase space that are visited by the trajectories. You can think of these as **recurrent regimes**. It is thought that at the regional scale, the atmospheric circulation 
follows a similar dynamic, with *N* (finite number of) attractor basins: regions of the phase space which are more likely than other to be visited, i.e. preferred **circulation regimes**, circulation 'types' or 'synoptic' types to refer to their spatial extent. 

The challenge is to determine these regimes from the data (e.g. daily 'maps' of geopotential)  

A common approach is to use [clustering](https://en.wikipedia.org/wiki/Cluster_analysis): i.e. extract N 'clusters' which group days which are 'close' to each other according to some distance criteria (e.g. but not limited to Euclidean distance), many methods exist to do that, from hierarchical clustering and non-hierarchical techniques.

### Relevance to historical climatology

+ day to day **weather** (daily temperatures, rainfall, pressure) related to weather regimes 


+ The occupation statistics of these regimes (frequency, persistence, preferred transitions) are modulated by large-scale climate modes (ENSO, SAM, etc)   


+ i.e. weather regimes provide the *link* between climate modes and local scale, daily *weather*  


+ can help to better understand non-linearities, assymetries in the regional signals of global climate modes  


+ can help 'reconstruct' likely circulation scenarios from a network of daily weather observations

<hr>

In this notebook we will try and replicate some of the results of the study by:  

**Kidson, 2000: An analysis of New Zealand synoptic types and their use in defining weather regimes, IJC, Volume 20, Issue 3, 15 March 2000, Pages 299–316**

who used [k-means](https://en.wikipedia.org/wiki/K-means_clustering) clustering to derive 12 synoptic types (The "Kidson Types") over the New Zealand region using reanalysed (NCEP / NCAR, *aka* NCEP 1) 1000 hPa geopotential from 1958 to 1997. 

The **Kidson types** have been used in a wide variety of studies (from subseasonal forecasting to paleoclimatology)

### import the stuff we need

In [ ]:
%matplotlib inline
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import xarray as xr

In [ ]:
from datetime import datetime, timedelta

In [ ]:
from glob import glob

In [ ]:
from IPython.display import Image

### downloading the daily NCEP / NCAR HGT from the ESRL and extract a spatial domain

as the NCEP / NCAR data is available via FTP rather than HTTP, we will use [cURL](https://curl.haxx.se/)  

You can download a **cURL** binary for your platform from [https://curl.haxx.se/dlwiz/](https://curl.haxx.se/dlwiz/)

#### base URL

In [ ]:
base_url = 'ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/'

#### Geographical domain and level to extract 

Be aware that the latitude variable is varying from North to South

In [ ]:
lonmin = 160
latmin = -55
lonmax = 185
latmax = -25
level = 1000

#### directory where to save the dataset

In [ ]:
opath = '../data/NZ/'

In [ ]:
# from subprocess import call
# for y in range(1958, 2016 + 1): 
#     filename = "hgt.{}.nc".format(y)
#     cmd = "curl --silent {}/{} -o {}/{}".format(base_url, filename, opath, filename)
#     r = call(cmd, shell=True)
#     if r != 0: 
#         print("something went wrong with the download of hgt.{}.nc".format(y))
#         pass
#     else: 
#         dset = xr.open_dataset('{}/{}'.format(opath, filename))
#         dset = dset.sel(lat=slice(latmax, latmin), lon=slice(lonmin,lonmax), level=level)
#         dset = dset.squeeze()
#         os.remove('{}/{}'.format(opath, filename))
#         dset.to_netcdf('{}/{}'.format(opath, filename))
#         dset.close()

### get the list of files in a Python list 

In [ ]:
lfiles = glob(os.path.join(opath, 'hgt*.nc')) 

In [ ]:
lfiles.sort()

In [ ]:
lfiles

### set a random seed to ensure reproducibility of the results 

In [ ]:
np.random.seed(42)

## reads in the dataset

In [ ]:
dset = xr.open_mfdataset(lfiles)

In [ ]:
dset

### select the period indicated to be utlized in the Kidson 2000 paper

In [ ]:
dset = dset.sel(time=slice('1958-1-1','1997-6-30'))

### calculates the time mean over the whole period

In [ ]:
dsetm = dset.mean('time')

### mapping with [cartopy](http://scitools.org.uk/cartopy/)

In [ ]:
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
lat = dset.lat
lon = dset.lon

In [ ]:
lons, lats = np.meshgrid(lon, lat)

In [ ]:
central_longitude = 180.

In [ ]:
proj = ccrs.PlateCarree(central_longitude=central_longitude)

In [ ]:
f, ax = plt.subplots(figsize=(10,8), subplot_kw={'projection':proj})

ax.coastlines('10m')

c = ax.contour(lons - central_longitude, lats, dsetm['hgt'], np.arange(-100, 200, 5))

plt.clabel(c, fmt='%i')

ax.set_extent([lon.data.min() - central_longitude, lon.data.max() - central_longitude, lat.data.min(), lat.data.max()], crs=proj)

### make a function 

In [ ]:
def make_map(X, lons, lats, vmin=-250, vmax=250, step=10, ax=None, central_longitude=180., fmt='%i'): 
    
    from numpy import ma
    
    if not(ax): 
        central_longitude = 180.
        proj = ccrs.PlateCarree(central_longitude=central_longitude)
        f, ax = plt.subplots(figsize=(10,8), subplot_kw={'projection':proj})
        
    proj = ccrs.PlateCarree(central_longitude=central_longitude)
    
    ax.coastlines('10m')
            
    if X.min() < 0 and X.max() > 0: 
        p = ax.contour(lons - central_longitude, lats, ma.masked_less(X,0), np.arange(0, vmax + step, step), colors='r')
        n = ax.contour(lons - central_longitude, lats, ma.masked_greater(X,0), np.arange(vmin, 0, step), colors='b')
        
        ax.contour(lons - central_longitude, lats, X, np.array([0]), colors='k')
        
        plt.clabel(p, fmt=fmt)
        plt.clabel(n, fmt=fmt) 
    elif X.min() < 0 and X.max() < 0: 
        n = ax.contour(lons - central_longitude, lats, X, np.arange(vmin, vmax + step, step), colors='b')
        plt.clabel(n, fmt=fmt)
    else: 
        p = ax.contour(lons - central_longitude, lats, X, np.arange(vmin, vmax + step, step), colors='r')
        plt.clabel(p, fmt=fmt)        
                            
    ax.set_extent([lon.data.min() - central_longitude, lon.data.max() - central_longitude, lat.data.min(), lat.data.max()], crs=proj)

In [ ]:
make_map(dsetm['hgt'].data, lons, lats)

In [ ]:
dset

In [ ]:
hgt = dset['hgt']

In [ ]:
hgt

### we now need to go from 3D (time, lat, lon) to 2D (time, space [lat X lon]) to perform the PCA (EOF) analyis

In [ ]:
hgt_stacked = hgt.stack(latlon=('lat', 'lon'))

In [ ]:
hgt_stacked

In [ ]:
hgt.shape

In [ ]:
hgt_stacked.shape

In [ ]:
type(hgt_stacked.data)

### we now need to LOAD the dataset in memory in order to perform the PCA (EOF analysis)

In [ ]:
hgt_stacked.load()

In [ ]:
type(hgt_stacked.data)

In [ ]:
X = hgt_stacked.data

In [ ]:
X.shape

In [ ]:
type(X)

The number of variables (features) is 143 (13 points in latitude * 11 points in longitude): while we can attend to perform our cluster analysis directly on this matrix, 
it is expensive computationally, i.e. each day (observation) is associated to a point with 143 coordinates, we need to find clusters in 143 dimensions ...

The approach generally taken is to **reduce the dimensionality** of the original dataset using methods such as [Principal Component (or Empirical Orthoginal Function) Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis). This is the approach that has been used by Kidson, and we will try and follow the way he processed the data as described in the paper.

## do PCA / EOF analysis of daily geopotential fields

To perform the PCA, we will use the excellent (not enough superlatives here !) [scikit-learn](http://scikit-learn.org/) Machine Learning library  

**Scikit-learn** implements a wide array of [supervised](http://en.wikipedia.org/wiki/Supervised_learning) and [unsupervised](http://en.wikipedia.org/wiki/Unsupervised_learning) Machine Learning algorithms, 

**Unsupervised** refers to algorithms which learn from a training set of unlabeled examples, using the features of the inputs to categorize inputs together according to some statistical criteria.

Unsupervised learning algorithms are usually separated into :

+ [Dimensionality reduction](http://scikit-learn.org/stable/modules/decomposition.html#decompositions) , whicn *learns* a more compact representation  of the data (i.e. reducing the dimensions).

+ [Clustering](http://scikit-learn.org/stable/modules/clustering.html#clustering)


scikit-learn exposes a very useful [preprocessing](http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing) submodule, which allows to do various transformations on your input data, in this case we will *standardisation* the dataset, removing the mean and dividing by the standard deviation ... 

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler  = StandardScaler()

In [ ]:
scaler = scaler.fit(X)

In [ ]:
X = scaler.transform(X)

In [ ]:
X.shape

In [ ]:
X.mean(0)

In [ ]:
X.mean()

In [ ]:
X.std(0)

### The pca class itself is in the *decomposition* submodule of scikit-learn

In [ ]:
from sklearn.decomposition import pca

In [ ]:
skpca = pca.PCA()

In [ ]:
skpca.fit(X)

In [ ]:
f, ax = plt.subplots(figsize=(6,6))
ax.plot(range(1,21), skpca.explained_variance_ratio_[0:20]*100)
ax.plot(range(1,21), skpca.explained_variance_ratio_[0:20]*100,'ro')
ax.grid(ls=':')
ax.set_xticks(range(1,21)); 
ax.set_xlabel('PC#');
ax.set_ylabel("% variance");

### In his paper, Kidson keeps the first 5 Principal Components and perform the cluster analysis on the subspace spanned by these 5 PCs

In [ ]:
ipc = 5

In [ ]:
skpca.explained_variance_ratio_[:ipc].sum()

Together they explain more than 90% of the total variance contained in the dataset

In [ ]:
PCs = skpca.transform(X)

In [ ]:
PCs = PCs[:,:ipc]

### the EOFS contain the spatial patterns associated with each PC

In [ ]:
EOFs = skpca.components_

In [ ]:
EOFs = EOFs[:ipc,:]

In [ ]:
EOFs.shape

In [ ]:
EOFs_r = EOFs.reshape((ipc, len(lat), len(lon)))

In [ ]:
make_map(EOFs_r[0,:,:], lons, lats, vmin=-0.2, vmax=0.2, step=0.010, fmt='%4.2f')

In his paper, not clear whether Kidson *scaled* (standardised) the PCs or not, trying with and without standardisation seem to indicated he DID NOT standardise the PCs prior to the cluster analysis

In [ ]:
# scaler_PCs = StandardScaler()
# scaler_PCs.fit(PCs)
# PCs_std = scaler_PCs.transform(PCs)

In [ ]:
PCdf = pd.DataFrame(PCs, index = dset['time'], \
                    columns = ["PC%s" % (x) for x in range(1, PCs.shape[1] +1)])

In [ ]:
PCdf.head()

In [ ]:
PCdf.tail()

In [ ]:
PCdf.plot(subplots=True, figsize=(12,10));

### The K-means clustering class is found in the [clustering](http://scikit-learn.org/stable/modules/clustering.html#clustering) submodule of scikit-learn

In [ ]:
from sklearn.cluster import KMeans

#### specify the number of clusters here ... 

In [ ]:
nclusters = 12

#### initialise the KMeans class with the parameters, n_jobs=-1 means the computations will be distributed accross the cores of your machine (if you have a multicore CPU)

In [ ]:
kmeans = KMeans(init='k-means++', n_clusters=nclusters, n_init=10, n_jobs=-1)

#### fit ... 

In [ ]:
kmeans.fit(PCdf.values)

#### `kmeans.labels_` contains the unique labels for each day: i.e. a number from 0 to nclusters-1 indicating to which cluster (regime) each day belongs

In [ ]:
kmeans.labels_

In [ ]:
np.unique(kmeans.labels_)

#### we put that into a Pandas DataFrame and assign the corresponding date to each day 

In [ ]:
labels = pd.DataFrame(kmeans.labels_, index=dset['time'], columns=['cluster'])

In [ ]:
labels.head()

In [ ]:
c = 0

In [ ]:
index = labels.query('cluster == {}'.format(c))

In [ ]:
nbdays = len(index)

In [ ]:
nbdays

In [ ]:
cluster = dset.sel(time=index.index).mean('time')

In [ ]:
cluster

In [ ]:
clusters = []
nbdays = []
for c in range(nclusters): 
    index = labels.query('cluster == {}'.format(c)) 
    nbdays.append(len(index))
    cluster = dset.sel(time=index.index).mean('time')
    clusters.append(cluster)

In [ ]:
clusters = xr.concat(clusters, dim='cluster')

In [ ]:
clusters

In [ ]:
f = clusters['hgt'].plot.contour(x='lon', y='lat', col='cluster', col_wrap=3, levels=np.arange(-150,200,20))

In [ ]:
sum(nbdays)

In [ ]:
f, axes = plt.subplots(nrows=3, ncols=4, figsize=(10,8), subplot_kw={'projection':proj})
f.subplots_adjust(wspace=0.1, hspace=0.1)
axes = axes.flatten() 
for c in range(nclusters): 
    ax = axes[c]
    clus = clusters.sel(cluster=c)
    make_map(clus['hgt'], lons, lats, step=25, ax=ax)
    ax.text(0.05, 0.9, "{}: {:3.2f}%".format(c, nbdays[c] / sum(nbdays) * 100), transform=ax.transAxes, bbox=dict(facecolor='w', alpha=0.5))

In [ ]:
f.savefig('./images/Kidson_clusters.png', dpi=200)

In [ ]:
!open ./images/Kidson_Archetypes.png

In [ ]:
Image('./images/Kidson_Archetypes.png', width=700)

### look at the seasonal distribution of the synoptic types / weather regimes

In [ ]:
f, axes = plt.subplots(nrows=4, ncols=3, figsize=(10,14))
axes = axes.flatten() 
for c in range(nclusters): 
    ax = axes[c]
    cf = labels.query('cluster == {}'.format(c))
    # in percentage
    ((cf.groupby(cf.index.month).count()) / len(cf) * 100).plot(kind='bar', width=1, ax=ax, legend=None)
    # in number of days
#     cf.groupby(cf.index.month).count().plot(kind='bar', width=1, ax=ax, legend=None)
    ax.set_ylim(0, None)
    ax.grid(ls=':')
    ax.text(0.05, 0.9, 'cluster {}'.format(c), transform=ax.transAxes, bbox=dict(facecolor='w', alpha=0.5))
    ax.set_xticklabels(list('JFMAMJJASOND'), rotation=0)
    ax.set_xlabel('')